![SolidQ](https://antoniosql.github.io/images/SolidQ_Verne.png) 

# Carga de datos y revisión

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt


In [2]:
conn = sqlite3.connect('Tiendas24H.sqlite')
consulta="select c.CodCliente, FechaNacimiento, EstadoCivil, Sexo, IngresosAnuales, N.NivelAcademico, O.Ocupacion, case     when v.CodCliente is null then 0   else 1  end as 'EsCliente' from clientes c inner join NivelAcademico N    on c.NivelAcademico = N.Codigo     inner join Ocupacion O on c.Ocupacion = o.Codigo     left outer join VentasCAB v          on v.codcliente = C.codcliente where c.Codcliente <>'CONTADO'"

df = pd.read_sql_query(consulta,conn)

df.head()

,CodCliente,FechaNacimiento,EstadoCivil,Sexo,IngresosAnuales,NivelAcademico,Ocupacion,EsCliente
0,000020,1976-04-08,M,M,90000.0,Licenciatura,Profesional especializado,1
1,000026,1975-05-14,S,M,60000.0,Licenciatura,Profesional especializado,1
2,000026,1975-05-14,S,M,60000.0,Licenciatura,Profesional especializado,1
3,000026,1975-05-14,S,M,60000.0,Licenciatura,Profesional especializado,1
4,000026,1975-05-14,S,M,60000.0,Licenciatura,Profesional especializado,1


In [3]:
df.to_json

<bound method NDFrame.to_json of       CodCliente FechaNacimiento EstadoCivil Sexo  IngresosAnuales  \
0         000020      1976-04-08           M    M          90000.0   
1         000026      1975-05-14           S    M          60000.0   
2         000026      1975-05-14           S    M          60000.0   
3         000026      1975-05-14           S    M          60000.0   
4         000026      1975-05-14           S    M          60000.0   
...          ...             ...         ...  ...              ...   
10393  Y1795344R      1944-09-17           M    F          70000.0   
10394  Y2404085R      1973-01-04           S    F          90000.0   
10395  Y2544794L      1973-08-26           M    M          90000.0   
10396  Y2671043K      1973-10-23           S    F          90000.0   
10397  Y2716961P      1973-03-16           M    M         110000.0   

                           NivelAcademico                  Ocupacion  \
0                            Licenciatura  Profesional

In [4]:
df.dropna(axis=0,inplace=True)

In [5]:
caracteristicas = ["EstadoCivil", "Sexo", "IngresosAnuales","NivelAcademico", "Ocupacion"]
etiqueta="EsCliente"

In [6]:
from sklearn.preprocessing import  LabelEncoder

le=LabelEncoder()
df['EstadoCivil'] = le.fit_transform(df['EstadoCivil'])
df['Sexo'] = le.fit_transform(df['Sexo'])

In [7]:
dum_df = pd.get_dummies(df[caracteristicas], columns=["NivelAcademico","Ocupacion"] )

In [8]:
from sklearn.model_selection import train_test_split
X, y = dum_df, df[etiqueta]

In [9]:
X_train, X_test, y_train, y_test =\
  train_test_split(X, y,
  test_size=0.3,
  random_state=0,
  stratify=y)

# Árboles de Decisión


In [10]:
from imblearn.over_sampling import SMOTE 
from collections import Counter

In [11]:
print('Distribución Original %s' % Counter(y))
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
print('Distribución balanceada %s' % Counter(y_res))

Distribución Original Counter({1: 10340, 0: 58})
Distribución balanceada Counter({1: 10340, 0: 10340})


In [12]:
from sklearn.model_selection import train_test_split
X_trainres, X_testres, y_trainres, y_testres = train_test_split(X_res, y_res, random_state=0)

In [13]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_trainres)
X_test = scaler.transform(X_testres)

In [14]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier().fit(X_trainres, y_trainres)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_trainres, y_trainres)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_testres, y_testres)))

Accuracy of Decision Tree classifier on training set: 0.97
Accuracy of Decision Tree classifier on test set: 0.97


In [16]:
X_testres.columns
#Columnas para construir la entrada 

Index(['EstadoCivil', 'Sexo', 'IngresosAnuales',
       'NivelAcademico_Educación Secundaria',
       'NivelAcademico_Educación Secundaria (en curso)',
       'NivelAcademico_Estudios de postgrado',
       'NivelAcademico_Estudios universitarios (en curso)',
       'NivelAcademico_Licenciatura', 'Ocupacion_Empleado por cuenta ajena',
       'Ocupacion_Empleados/as de hogar', 'Ocupacion_Mando intermedio',
       'Ocupacion_Profesional especializado',
       'Ocupacion_Profesional independiente'],
      dtype='object')

# Serializar el Modelo

In [15]:
import os
import pickle

fichero = 'clf'
pickle.dump(clf, open(fichero, 'wb'))
#load_model = pickle.load(open(filename, 'rb'))
#result = load_model.score(X_testres, y_testres)

# Código Servidor para Publicar API


In [16]:
import numpy as np
from flask import Flask, request, jsonify
import pickle


In [17]:
load_model = pickle.load(open(fichero, 'rb'))

In [18]:

app = Flask(__name__)

@app.route('/api',methods=['POST'])
def predict():
    data = request.get_json(force=True)
   
    predict_request=[[data['EstadoCivil'],data['Sexo'],data['IngresosAnuales'],data['NivelAcademico_Educación Secundaria'], data['NivelAcademico_Educación Secundaria (en curso)'], data['NivelAcademico_Estudios de postgrado'],data['NivelAcademico_Estudios universitarios (en curso)'],data['NivelAcademico_Licenciatura'], data['Ocupacion_Empleado por cuenta ajena'],data['Ocupacion_Empleados/as de hogar'],data['Ocupacion_Mando intermedio'],data['Ocupacion_Profesional especializado'],data['Ocupacion_Profesional independiente']]] #Construir Datos Petición
    
   
    peticion=np.array(predict_request)
   
    prediction = load_model.predict(peticion)
   
    pred = prediction[0]
   
    return jsonify(int(pred))

if __name__ == '__main__':
    #from werkzeug.serving import run_simple
    #app.run(port=9000, debug=True)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2020 16:50:36] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2020 16:51:15] "POST /api HTTP/1.1" 200 -
